# Neural Nets from Scratch in Julia

## Lesson 8: Negation and subtraction

* In this video we'll implement negation and subtraction for *Values*, and do this in a clever way so that it only requires a forward pass without needing its own backward pass.
* [Documentation site here](https://mikesaint-antoine.github.io/SimpleGrad.jl)
* [Github repo here](https://github.com/mikesaint-antoine/SimpleGrad.jl)

In [1]:
## code so far

mutable struct Value{opType} <: Number
    data::Float64
    grad::Float64
    op::opType
end


struct Operation{FuncType, ArgTypes}
    op::FuncType
    args::ArgTypes
end

# constructor -- Value(data, grad, op)
Value(x::Number) = Value(Float64(x), 0.0, nothing);


import Base.show
function show(io::IO, value::Value)
    print(io, "Value(",value.data,")")
end


import Base.==
function ==(a::Value, b::Value)
     return a===b
end


import Base.+
function +(a::Value, b::Value)

    out = a.data + b.data    
    result = Value(out, 0.0, Operation(+, (a,b))) # Value(data, grad, op)
    return result # this should be a Value
 
end



backprop!(val::Value{Nothing}) = nothing


function backprop!(val::Value{Operation{FunType, ArgTypes}}) where {FunType<:typeof(+), ArgTypes}
    
    # val = a + b
    # update a.grad, b.grad
    
    val.op.args[1].grad += val.grad
    val.op.args[2].grad += val.grad
    
end




function backward(a::Value)
    
    
    function build_topo(v::Value, visited=Value[], topo=Value[])
    
        if !(v in visited)
            
            push!(visited, v)
            
            if v.op != nothing
                for operand in v.op.args
                    
                    if operand isa Value
                        build_topo(operand, visited, topo)
                    end
                end 
            end
            
            push!(topo, v) 
            
            
        end
        return topo
    end
    
    
    
    topo = build_topo(a)
    
    a.grad = 1
    #da/da = 1
    
    for node in reverse(topo)
        backprop!(node)
    end
    
    
end


Base.promote_rule(::Type{<:Value}, ::Type{T}) where {T<:Number} = Value




import Base.*
function *(a::Value, b::Value)

    out = a.data * b.data    
    result = Value(out, 0.0, Operation(*, (a,b))) # Value(data, grad, op)
    return result # this should be a Value
 
end



function backprop!(val::Value{Operation{FunType, ArgTypes}}) where {FunType<:typeof(*), ArgTypes}
    
    # val = a * b
    # update a.grad, b.grad
    
    val.op.args[1].grad += val.op.args[2].data * val.grad    
    val.op.args[2].grad += val.op.args[1].data * val.grad
    
end


backprop! (generic function with 3 methods)

In [4]:
import Base.-

# negation
function -(a::Value)
    
    return a * -1
    
end

# subtraction
function -(a::Value, b::Value)
    
    return a + (-b)
    
    
end


- (generic function with 199 methods)

In [5]:
a = Value(1)
b = Value(2)
c = Value(3)
d = Value(4)


y = a*b - c*d

backward(y)

println(a.grad)
# dy/da = b = 2

println(b.grad)
# dy/db = a = 1

println(c.grad)
# dy/dc = -d = -4

println(d.grad)
# dy/dd = -c = -3


2.0
1.0
-4.0
-3.0
